In [287]:
import pandas as pd
import os
from datetime import datetime, timedelta
import csv
import numpy as np

# Stress Tests

## Responsiveness

In [288]:
projectPath = os.path.abspath(os.path.join(os.getcwd(), '..'))
mode_dir_1 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_1/social_groups')
mode_dir_2 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_2/social_groups')
mode_dir_3 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_3/social_groups')
output_name_1 = 'responsiveness_stress_1'
output_name_2 = 'responsiveness_stress_2'
output_name_3 = 'responsiveness_stress_3'

def compute_responsiveness_stress_test(projectPath, mode_dir, output_name):
    records = []
    # Traverse each folder inside the mode
    for date in os.listdir(mode_dir):
        date_path = os.path.join(mode_dir, date)
        if not os.path.isdir(date_path):
            continue
        summary_path = os.path.join(date_path, 'simulation_summary.csv')
        if os.path.exists(summary_path):
            df_summary = pd.read_csv(summary_path)
            if not df_summary.empty:
                # Get time metrics
                first_row = df_summary.iloc[0]
                elapsed_sec = first_row['total_elapsed_seconds']
                sumo_sec = first_row['sumo_time']
                agents_sec = first_row['agents_time']
                metrics_path = os.path.join(date_path, 'sf_final_metrics.csv')
                if os.path.exists(metrics_path):
                    df_metrics = pd.read_csv(metrics_path)
                    if not df_metrics.empty:
                        records.append({
                            'mode': 'social_groups',
                            'date': date,
                            'elapsed_seconds': elapsed_sec,
                            'elapsed_minutes': round(elapsed_sec / 60, 2),
                            'sumo_time': sumo_sec,
                            'sumo_minutes': round(sumo_sec / 60, 2),
                            'agents_time': agents_sec,
                            'agents_minutes': round(agents_sec / 60, 2),
                            'sumo_perc_time': round(sumo_sec / elapsed_sec, 2),
                            'agents_perc_time': round(agents_sec / elapsed_sec, 2) if agents_sec > 0 else 0
                        })
                        
    summary_df = pd.DataFrame(records)
    output_file = os.path.join(projectPath, f'experiments/results/{output_name}.csv')
    summary_df = summary_df.sort_values(by=['date'])
    summary_df.to_csv(output_file, index=False)
    print(f"Responsiveness summary saved to: {output_file}")


# Compute responsiveness for each stress test
compute_responsiveness_stress_test(projectPath, mode_dir_1, output_name_1)
compute_responsiveness_stress_test(projectPath, mode_dir_2, output_name_2)
compute_responsiveness_stress_test(projectPath, mode_dir_3, output_name_3)

Responsiveness summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/responsiveness_stress_1.csv
Responsiveness summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/responsiveness_stress_2.csv
Responsiveness summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/responsiveness_stress_3.csv


## Fidelity

In [289]:
def get_pickups_dropoffs(
        sf_rides_stats_path,
        start_date_str,
        start_time_str,
        end_date_str,
        end_time_str,
        stress_rate
    ):
    start_date = datetime.strptime(start_date_str, "%y%m%d").date()
    end_date = datetime.strptime(end_date_str, "%y%m%d").date()
    num_days = (end_date - start_date).days + 1
    start_hour = int(datetime.strptime(start_time_str, "%H").hour)
    end_hour = int(datetime.strptime(end_time_str, "%H").hour)

    # Map dataset hours (3–26) to standard 0–23 format
    dataset_hour_map = {h: h % 24 for h in range(3, 27)}

    all_rows = []
    with open(sf_rides_stats_path, mode='r') as file:
        reader = csv.DictReader(file, delimiter=',')
        for row in reader:
            row['taz'] = int(row['taz'])
            row['day_of_week'] = int(row['day_of_week'])
            row['hour'] = int(row['hour'])
            row['pickups'] = round(float(row['pickups']))
            row['dropoffs'] = round(float(row['dropoffs']))
            all_rows.append(row)

    # Index by (day_of_week, hour, taz)
    data_by_key = {}
    for row in all_rows:
        key = (row['day_of_week'], row['hour'], row['taz'])
        data_by_key[key] = {'pickups': row['pickups'], 'dropoffs': row['dropoffs']}

    zone_data = {}
    # For each simulation day, determine hours to include from that day
    for sim_day_index in range(num_days):
        sim_date = start_date + timedelta(days=sim_day_index)
        sim_day_of_week = sim_date.weekday()
        if num_days == 1:
            selected_std_hours = list(range(start_hour, end_hour))
        else:
            if sim_day_index == 0:
                selected_std_hours = list(range(start_hour, 24))
            elif sim_day_index == num_days - 1:
                selected_std_hours = list(range(0, end_hour))
            else:
                selected_std_hours = list(range(0, 24))
        selected_dataset_hours = {h: std for h, std in dataset_hour_map.items() if std in selected_std_hours}
        # Filter rows for this day and hour
        for row in all_rows:
            taz = row['taz']
            hour = row['hour']
            day = row['day_of_week']
            if day == sim_day_of_week and hour in selected_dataset_hours:
                std_hour = selected_dataset_hours[hour]
                if taz not in zone_data:
                    zone_data[taz] = {}
                zone_data[taz][std_hour] = {
                    'pickups': row['pickups'],
                    'dropoffs': row['dropoffs']
                }
    
    # If tazs_involved is provided, adjust pickups and dropoffs based on stress test conditions
    for taz in zone_data:
        for hour in zone_data[taz]:
            zone_data[taz][hour]['pickups'] = round(zone_data[taz][hour]['pickups'] * (1+stress_rate))
            zone_data[taz][hour]['dropoffs'] = round(zone_data[taz][hour]['dropoffs'] * (1+stress_rate))

    # Compute pickups and dropoffs across all zones and selected hours
    total_pickups = sum(hour_data['pickups'] for zone in zone_data.values() for hour_data in zone.values())
    total_dropoffs = sum(hour_data['dropoffs'] for zone in zone_data.values() for hour_data in zone.values())

    return total_pickups, total_dropoffs


def percent_error(true_val, estimated_val):
    return 100 * abs(true_val - estimated_val) / true_val if true_val != 0 else float('nan')

In [290]:
projectPath = os.path.abspath(os.path.join(os.getcwd(), '..'))
mode_dir_1 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_1/social_groups')
mode_dir_2 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_2/social_groups')
mode_dir_3 = os.path.join(projectPath, 'sumoenv/scenarios/stress_test_3/social_groups')
output_name_1 = 'fidelity_stress_1'
output_name_2 = 'fidelity_stress_2'
output_name_3 = 'fidelity_stress_3'

def compute_geh(observed, modeled):
    return np.sqrt(2 * (observed - modeled)**2 / (observed + modeled + 1e-6))

def compute_fidelity_stress_test(projectPath, mode_dir, output_name, stress_rate):
    traffic_dir = os.path.join(projectPath, 'data/sf_traffic/sfmta_dataset')
    sfcta_dir = os.path.join(projectPath, 'data/ridehailing_stats')
    # Initialize records and TAZs involved
    records = []
    # Traverse each folder inside the mode
    for date in os.listdir(mode_dir):
        start, end = date.split('_')
        start_date_str = "20" + start[:6]
        start_time_str = start[6:] + "00"
        end_date_str = "20" + end[:6]
        end_time_str = end[6:] + "00"
        date_path = os.path.join(mode_dir, date)
        if not os.path.isdir(date_path):
            continue
        summary_path = os.path.join(date_path, 'sf_final_metrics.csv')
        # Get real traffic data
        traffic_file = [f for f in os.listdir(traffic_dir) if os.path.isfile(os.path.join(traffic_dir, f)) and date in f]
        traffic_df = pd.read_csv(os.path.join(traffic_dir, traffic_file[0]))
        traffic_df["vehicle_position_date_time"] = pd.to_datetime(traffic_df["vehicle_position_date_time"])
        start_dt = datetime.strptime(f"{start_date_str} {start_time_str}", "%Y%m%d %H%M")
        end_dt = datetime.strptime(f"{end_date_str} {end_time_str}", "%Y%m%d %H%M")
        traffic_df = traffic_df[(traffic_df["vehicle_position_date_time"] >= start_dt) & (traffic_df["vehicle_position_date_time"] < end_dt)]
        # Filter out vehicles that have speed == 0 for all their records
        valid_vehicle_ids = traffic_df.groupby("vehicle_id")["average_speed"].max()
        valid_vehicle_ids = valid_vehicle_ids[valid_vehicle_ids > 0].index
        traffic_df = traffic_df[traffic_df["vehicle_id"].isin(valid_vehicle_ids)]
        added_count = (len(traffic_df)-1) * stress_rate
        traffic = (len(traffic_df)-1) + added_count
        # Scale traffic: 36% of traffic is TNC
        traffic_scaled = int((traffic) * 0.64)
        # Get pickups and dropoffs data
        start_str, end_str = date.split('_')
        pickups, dropoffs = get_pickups_dropoffs(
            os.path.join(sfcta_dir, "trip_stats_taz.csv"),
            start_str[:6],
            start_str[6:],
            end_str[:6],
            end_str[6:],
            stress_rate 
        )
        # Compute errors and record results
        if os.path.exists(summary_path):
            df = pd.read_csv(summary_path)
            if not df.empty:
                traffic_count = sum(df['traffic_departures'])
                pickup_count = sum(df['passengers_departures'])
                dropoff_count = sum(df['passengers_arrivals'])
                canceled_count = sum(df['passengers_cancel']) + sum(df['rides_not_served'])
                traffic_error = percent_error(traffic_scaled, traffic_count)
                pickup_error = percent_error(pickups, pickup_count)
                dropoff_error = percent_error(dropoffs, dropoff_count)
                pickup_scaled_error = percent_error(pickups, pickup_count + canceled_count)
                dropoff_scaled_error = percent_error(dropoffs, dropoff_count + canceled_count)
                # Compute GEH hourly values
                if end_time_str in ["0900", "2100"]:
                    geh_traffic = compute_geh(traffic_scaled, traffic_count)
                    geh_pickup = compute_geh(pickups, pickup_count + canceled_count)
                    geh_dropoff = compute_geh(dropoffs, dropoff_count + canceled_count)
                if end_time_str in ["1100", "2300"]:
                    geh_traffic = compute_geh(traffic_scaled/3, traffic_count/3)
                    geh_pickup = compute_geh(pickups/3, (pickup_count + canceled_count)/3)
                    geh_dropoff = compute_geh(dropoffs/3, (dropoff_count + canceled_count)/3)
                if end_time_str in ["1400", "0200"]:
                    geh_traffic = compute_geh(traffic_scaled/6, traffic_count/6)
                    geh_pickup = compute_geh(pickups/6, (pickup_count + canceled_count)/6)
                    geh_dropoff = compute_geh(dropoffs/6, (dropoff_count + canceled_count)/6)
                if end_time_str in ["2000", "0800"]:
                    geh_traffic = compute_geh(traffic_scaled/12, traffic_count/12)
                    geh_pickup = compute_geh(pickups/12, (pickup_count + canceled_count)/12)
                    geh_dropoff = compute_geh(dropoffs/12, (dropoff_count + canceled_count)/12)
                records.append({
                    'mode': 'social_groups',
                    'date': date,
                    'traffic_input': traffic_scaled,
                    'pickup_input': pickups,
                    'dropoff_input': dropoffs,
                    'traffic_output': traffic_count,
                    'pickup_output': pickup_count,
                    'dropoff_output': dropoff_count,
                    'traffic_error': round(traffic_error, 2),
                    'pickup_error': round(pickup_error, 2),
                    'dropoff_error': round(dropoff_error, 2),
                    'canceled_rides': canceled_count,
                    'pickup_scaled_error': round(pickup_scaled_error, 2),
                    'dropoff_scaled_error': round(dropoff_scaled_error, 2),
                    'geh_traffic': round(geh_traffic, 2),
                    'geh_pickup': round(geh_pickup, 2),
                    'geh_dropoff': round(geh_dropoff, 2)
                })

    summary_df = pd.DataFrame(records)
    output_file = os.path.join(projectPath, f'experiments/results/{output_name}.csv')
    summary_df = summary_df.sort_values(by=['date'])
    summary_df.to_csv(output_file, index=False)
    print(f"Fidelity summary saved to: {output_file}")

# Compute fidelity for each stress test
compute_fidelity_stress_test(projectPath, mode_dir_1, output_name_1, stress_rate=0.25)
compute_fidelity_stress_test(projectPath, mode_dir_2, output_name_2, stress_rate=0.5)
compute_fidelity_stress_test(projectPath, mode_dir_3, output_name_3, stress_rate=1.0)

Fidelity summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/fidelity_stress_1.csv
Fidelity summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/fidelity_stress_2.csv
Fidelity summary saved to: /Users/beyzaeken/Desktop/sfdigitalmirror/experiments/results/fidelity_stress_3.csv


## Responsiveness

In [296]:
# Compare responsiveness of stress tests with normal scenario
df_stress_1_responsiveness = pd.read_csv(os.path.join(projectPath, 'experiments/results/responsiveness_stress_1.csv'))
df_stress_2_responsiveness = pd.read_csv(os.path.join(projectPath, 'experiments/results/responsiveness_stress_2.csv'))
df_stress_3_responsiveness = pd.read_csv(os.path.join(projectPath, 'experiments/results/responsiveness_stress_3.csv'))
df_normal_responsiveness = pd.read_csv(os.path.join(projectPath, 'experiments/results/responsiveness_normal.csv'))
df_stress_dates = df_stress_1_responsiveness['date'].unique()
df_normal_responsiveness = df_normal_responsiveness[df_normal_responsiveness['mode'] == 'social_groups_avg']
df_normal_responsiveness = df_normal_responsiveness[df_normal_responsiveness['date'].isin(df_stress_dates)]
normal_seconds = df_normal_responsiveness['elapsed_seconds'].mean()
normal_agents = df_normal_responsiveness['agents_time'].mean()
normal_agents_perc = (df_normal_responsiveness['agents_perc_time'].mean() * 100)
stress_1_seconds = df_stress_1_responsiveness['elapsed_seconds'].mean()
stress_1_agents = df_stress_1_responsiveness['agents_time'].mean()
stress_1_agents_perc = (df_stress_1_responsiveness['agents_perc_time'].mean() * 100)
stress_2_seconds = df_stress_2_responsiveness['elapsed_seconds'].mean()
stress_2_agents = df_stress_2_responsiveness['agents_time'].mean()
stress_2_agents_perc = (df_stress_2_responsiveness['agents_perc_time'].mean() * 100)
stress_3_seconds = df_stress_3_responsiveness['elapsed_seconds'].mean()
stress_3_agents = df_stress_3_responsiveness['agents_time'].mean()
stress_3_agents_perc = (df_stress_3_responsiveness['agents_perc_time'].mean() * 100)
ratio_1 = ((stress_1_seconds - normal_seconds) / normal_seconds) * 100
ratio_2 = ((stress_2_seconds - normal_seconds) / normal_seconds) * 100
ratio_3 = ((stress_3_seconds - normal_seconds) / normal_seconds) * 100
print(f"Normal responsiveness: {normal_seconds:.2f} seconds")
print(f"Stress test 1 responsiveness: {stress_1_seconds:.2f} seconds")
print(f"Responsiveness ratio (stress_test_1/normal): {ratio_1:.2f}%")
print(f"Stress test 2 responsiveness: {stress_2_seconds:.2f} seconds")
print(f"Responsiveness ratio (stress_test_2/normal): {ratio_2:.2f}%")
print(f"Stress test 3 responsiveness: {stress_3_seconds:.2f} seconds")
print(f"Responsiveness ratio (stress_test_3/normal): {ratio_3:.2f}%\n")
print(f"Normal agents responsiveness: {normal_agents:.2f} ({normal_agents_perc:.2f}%)")
print(f"Stress test 1 agents responsiveness: {stress_1_agents:.2f} ({stress_1_agents_perc:.2f}%)")
print(f"Stress test 2 agents responsiveness: {stress_2_agents:.2f} ({stress_2_agents_perc:.2f}%)")
print(f"Stress test 3 agents responsiveness: {stress_3_agents:.2f} ({stress_3_agents_perc:.2f}%)\n")
print(f"Normal sumo responsiveness: {normal_seconds - normal_agents:.2f} ({100 - normal_agents_perc:.2f}%)")
print(f"Stress test 1 sumo responsiveness: {stress_1_seconds - stress_1_agents:.2f} ({100 - stress_1_agents_perc:.2f}%)")
print(f"Stress test 2 sumo responsiveness: {stress_2_seconds - stress_2_agents:.2f} ({100 - stress_2_agents_perc:.2f}%)")
print(f"Stress test 3 sumo responsiveness: {stress_3_seconds - stress_3_agents:.2f} ({100 - stress_3_agents_perc:.2f}%)")

Normal responsiveness: 962.08 seconds
Stress test 1 responsiveness: 1562.83 seconds
Responsiveness ratio (stress_test_1/normal): 62.44%
Stress test 2 responsiveness: 2296.06 seconds
Responsiveness ratio (stress_test_2/normal): 138.66%
Stress test 3 responsiveness: 3831.32 seconds
Responsiveness ratio (stress_test_3/normal): 298.23%

Normal agents responsiveness: 421.15 (44.33%)
Stress test 1 agents responsiveness: 655.69 (43.22%)
Stress test 2 agents responsiveness: 900.43 (41.33%)
Stress test 3 agents responsiveness: 1247.77 (32.25%)

Normal sumo responsiveness: 540.93 (55.67%)
Stress test 1 sumo responsiveness: 907.15 (56.78%)
Stress test 2 sumo responsiveness: 1395.63 (58.67%)
Stress test 3 sumo responsiveness: 2583.55 (67.75%)


## Fidelity

In [298]:
# Compare fidelity of stress tests with normal scenario
df_stress_1_fidelity = pd.read_csv(os.path.join(projectPath, 'experiments/results/fidelity_stress_1.csv'))
df_stress_2_fidelity = pd.read_csv(os.path.join(projectPath, 'experiments/results/fidelity_stress_2.csv'))
df_stress_3_fidelity = pd.read_csv(os.path.join(projectPath, 'experiments/results/fidelity_stress_3.csv'))
df_normal_fidelity = pd.read_csv(os.path.join(projectPath, 'experiments/results/fidelity_normal.csv'))
df_stress_dates = df_stress_1_fidelity['date'].unique()
df_normal_fidelity = df_normal_fidelity[df_normal_fidelity['mode'] == 'social_groups_avg']
df_normal_fidelity = df_normal_fidelity[df_normal_fidelity['date'].isin(df_stress_dates)]
normal_traffic_acc = 100 - df_normal_fidelity['traffic_error'].mean()
normal_pickup_acc = 100 - df_normal_fidelity['pickup_scaled_error'].mean()
normal_dropoff_acc = 100 - df_normal_fidelity['dropoff_scaled_error'].mean()
stress_1_traffic_acc, min_stress_1_traffic, max_stress_1_traffic = 100 - df_stress_1_fidelity['traffic_error'].mean(), 100 - df_stress_1_fidelity['traffic_error'].max(), 100 - df_stress_1_fidelity['traffic_error'].min()
stress_1_pickup_acc, min_stress_1_pickup, max_stress_1__pickup = 100 - df_stress_1_fidelity['pickup_scaled_error'].mean(), 100 - df_stress_1_fidelity['pickup_scaled_error'].max(), 100 - df_stress_1_fidelity['pickup_scaled_error'].min()
stress_1_dropoff_acc, min_stress_1_dropoff, max_stress_1_dropoff = 100 - df_stress_1_fidelity['dropoff_scaled_error'].mean(), 100 - df_stress_1_fidelity['dropoff_scaled_error'].max(), 100 - df_stress_1_fidelity['dropoff_scaled_error'].min()
stress_2_traffic_acc, min_stress_2_traffic, max_stress_2_traffic = 100 - df_stress_2_fidelity['traffic_error'].mean(), 100 - df_stress_2_fidelity['traffic_error'].max(), 100 - df_stress_2_fidelity['traffic_error'].min()
stress_2_pickup_acc, min_stress_2_pickup, max_stress_2_pickup = 100 - df_stress_2_fidelity['pickup_scaled_error'].mean(), 100 - df_stress_2_fidelity['pickup_scaled_error'].max(), 100 - df_stress_2_fidelity['pickup_scaled_error'].min()
stress_2_dropoff_acc, min_stress_2_dropoff, max_stress_2_dropoff = 100 - df_stress_2_fidelity['dropoff_scaled_error'].mean(), 100 - df_stress_2_fidelity['dropoff_scaled_error'].max(), 100 - df_stress_2_fidelity['dropoff_scaled_error'].min()
stress_3_traffic_acc, min_stress_3_traffic, max_stress_3_traffic = 100 - df_stress_3_fidelity['traffic_error'].mean(), 100 - df_stress_3_fidelity['traffic_error'].max(), 100 - df_stress_3_fidelity['traffic_error'].min()
stress_3_pickup_acc, min_stress_3_pickup, max_stress_3_pickup = 100 - df_stress_3_fidelity['pickup_scaled_error'].mean(), 100 - df_stress_3_fidelity['pickup_scaled_error'].max(), 100 - df_stress_3_fidelity['pickup_scaled_error'].min()
stress_3_dropoff_acc, min_stress_3_dropoff, max_stress_3_dropoff = 100 - df_stress_3_fidelity['dropoff_scaled_error'].mean(), 100 - df_stress_3_fidelity['dropoff_scaled_error'].max(), 100 - df_stress_3_fidelity['dropoff_scaled_error'].min()
traffic_ratio_1 = ((stress_1_traffic_acc - normal_traffic_acc) / normal_traffic_acc) * 100
pickup_ratio_1 = ((stress_1_pickup_acc - normal_pickup_acc) / normal_pickup_acc) * 100
dropoff_ratio_1 = ((stress_1_dropoff_acc - normal_dropoff_acc) / normal_dropoff_acc) * 100
traffic_ratio_2 = ((stress_2_traffic_acc - normal_traffic_acc) / normal_traffic_acc) * 100
pickup_ratio_2 = ((stress_2_pickup_acc - normal_pickup_acc) / normal_pickup_acc) * 100
dropoff_ratio_2 = ((stress_2_dropoff_acc - normal_dropoff_acc) / normal_dropoff_acc) * 100
traffic_ratio_3 = ((stress_3_traffic_acc - normal_traffic_acc) / normal_traffic_acc) * 100
pickup_ratio_3 = ((stress_3_pickup_acc - normal_pickup_acc) / normal_pickup_acc) * 100
dropoff_ratio_3 = ((stress_3_dropoff_acc - normal_dropoff_acc) / normal_dropoff_acc) * 100
print(f"Traffic accuracy normal: {normal_traffic_acc:.2f}%")
print(f"Pickup accuracy normal: {normal_pickup_acc:.2f}%")
print(f"Dropoff accuracy normal: {normal_dropoff_acc:.2f}%")
print(f"Traffic accuracy stress 1: {stress_1_traffic_acc:.2f}%, min: {min_stress_1_traffic:.2f}%, max: {max_stress_1_traffic:.2f}%")
print(f"Pickup accuracy stress 1: {stress_1_pickup_acc:.2f}%, min: {min_stress_1_pickup:.2f}%, max: {max_stress_1__pickup:.2f}%")
print(f"Dropoff accuracy stress 1: {stress_1_dropoff_acc:.2f}%, min: {min_stress_1_dropoff:.2f}%, max: {max_stress_1_dropoff:.2f}%")
print(f"Traffic accuracy stress 2: {stress_2_traffic_acc:.2f}%, min: {min_stress_2_traffic:.2f}%, max: {max_stress_2_traffic:.2f}%")
print(f"Pickup accuracy stress 2: {stress_2_pickup_acc:.2f}%, min: {min_stress_2_pickup:.2f}%, max: {max_stress_2_pickup:.2f}%")
print(f"Dropoff accuracy stress 2: {stress_2_dropoff_acc:.2f}%, min: {min_stress_2_dropoff:.2f}%, max: {max_stress_2_dropoff:.2f}%")
print(f"Traffic accuracy stress 3: {stress_3_traffic_acc:.2f}%, min: {min_stress_3_traffic:.2f}%, max: {max_stress_3_traffic:.2f}%")
print(f"Pickup accuracy stress 3: {stress_3_pickup_acc:.2f}%, min: {min_stress_3_pickup:.2f}%, max: {max_stress_3_pickup:.2f}%")
print(f"Dropoff accuracy stress 3: {stress_3_dropoff_acc:.2f}%, min: {min_stress_3_dropoff:.2f}%, max: {max_stress_3_dropoff:.2f}%\n")
print(f"Traffic accuracy ratio (stress_test_1/normal): {traffic_ratio_1:.2f}%")
print(f"Pickup accuracy ratio (stress_test_1/normal): {pickup_ratio_1:.2f}%")
print(f"Dropoff accuracy ratio (stress_test_1/normal): {dropoff_ratio_1:.2f}%")
print(f"Traffic accuracy ratio (stress_test_2/normal): {traffic_ratio_2:.2f}%")
print(f"Pickup accuracy ratio (stress_test_2/normal): {pickup_ratio_2:.2f}%")
print(f"Dropoff accuracy ratio (stress_test_2/normal): {dropoff_ratio_2:.2f}%")
print(f"Traffic accuracy ratio (stress_test_3/normal): {traffic_ratio_3:.2f}%")
print(f"Pickup accuracy ratio (stress_test_3/normal): {pickup_ratio_3:.2f}%")
print(f"Dropoff accuracy ratio (stress_test_3/normal): {dropoff_ratio_3:.2f}%")

Traffic accuracy normal: 99.79%
Pickup accuracy normal: 99.90%
Dropoff accuracy normal: 99.82%
Traffic accuracy stress 1: 99.22%, min: 97.38%, max: 99.92%
Pickup accuracy stress 1: 99.85%, min: 99.75%, max: 99.97%
Dropoff accuracy stress 1: 99.79%, min: 99.71%, max: 99.88%
Traffic accuracy stress 2: 97.46%, min: 93.16%, max: 99.80%
Pickup accuracy stress 2: 99.85%, min: 99.69%, max: 99.99%
Dropoff accuracy stress 2: 99.73%, min: 99.57%, max: 99.93%
Traffic accuracy stress 3: 93.61%, min: 84.65%, max: 99.59%
Pickup accuracy stress 3: 99.87%, min: 99.77%, max: 100.00%
Dropoff accuracy stress 3: 99.78%, min: 99.59%, max: 99.96%

Traffic accuracy ratio (stress_test_1/normal): -0.57%
Pickup accuracy ratio (stress_test_1/normal): -0.05%
Dropoff accuracy ratio (stress_test_1/normal): -0.03%
Traffic accuracy ratio (stress_test_2/normal): -2.33%
Pickup accuracy ratio (stress_test_2/normal): -0.05%
Dropoff accuracy ratio (stress_test_2/normal): -0.09%
Traffic accuracy ratio (stress_test_3/normal